# E-commerce Algorithmic Analysis


In my attempt to simulate the start of an e-commerce business, I generated 100 million products/sku's representing potential items to sell. From this data, I extract 2 million. For each item I generate a single sellability column that I envision to be correlated to profitability - competitiveness + popularity.

Profitability represents the amount each product can be sold for, minus the cost to buy and import. Competitiveness represents a measure of the number of existing similar products with favorable reviews and fewer complaints. Popularity of a product measures the quantity of recent mentions across social media and other online venues.

I generate categories/subcategories (generated by ChatGPT) for each item.

  
Tasks: 

1. Check for duplicates in data by uploading data into a pandas dataframe. Check using a naive nested loop on the column: O(n^2). More efficiently, make a "set" and return any entries that are duplicates: O(n). 

2. With a dictionary I can do an SKU lookup in O(1).

3. Break the data by category, create a priority queue Minheap (O(n)) that maximizes sellability. Within each category, sort each heap for a total time complexity of O(nlogn).


# Preparation
- import libraries
- import file with 2 million data entries 
- generate an array of SKU column for later use
- generate a dictionary that maps each SKU to corresponding row for later use

In [16]:
import pandas as pd
import numpy as np
import time
from numba import njit
import random
import heapq

df = pd.read_csv("generated_data.txt").iloc[:2_000_000]
df.columns = df.columns.str.replace(" ", "", regex=False) ## credit chatGPT removing spaces from columns 
df_numpy = df["SKU"].to_numpy()

# generate dictionary (preparation for the O(1) algorithm) 
# Dictionary stores all the SKU's and maps to the corresponding row
dict_1 = {}
temp_columns = df.columns
for i in range(len(df)):
    temp_row = df.iloc[i]
    if temp_row["SKU"] not in dict_1:
        dict_1[temp_row["SKU"]] = {j : temp_row[j] for j in temp_columns[1:]}

# O(1)Algorithm


In [17]:
# O(1) algorithm 
#Dictionary lookup for a given SKU
def dict_lookup(SKU, dict = dict_1):
    # generate random_SKU
    if SKU in dict:
        return dict[SKU]

# O(n) Algorithm


In [18]:
#Duplicate search
def duplicate_search_set(data):
    #create a set
    s = set()
    #create a counter for number of duplicates
    duplicates = 0
    # create a list for list of duplicates
    duplicate_list = []
    #iterate through data and check against set
    for i in data:
        # Check if already in set
        if i in s:
            duplicates += 1
            duplicate_list.append(i)
        # Add element to set
        s.add(i)
    # return counter, duplicate list
    return duplicates, duplicate_list

# O(nlogn) Algorithm 

In [19]:
def heapify_categories(dataframe = df, column = "Category"):

    # create a dictionary that maps a category to a dataframe
    category_dict = {i:j for i,j in dataframe.groupby("Category")}
 
    #create dictionary mapping category to heap of rows in that category
    heap_dict = {}
    #create a second dictionary mapping category to sorted heap of rows in that category
    heap_dict2 = {}

    for category in category_dict: 
        # Create a list to store the heap
        heap_list = []
        # Create temp df that stores category dataframe 
        temp_df = category_dict[category]
        # store each row from category-dataframe as a tuple with Sellability coming first (for heap)
        for row in temp_df.itertuples():
            #get index as a tie-breaker when comparing two items with same Sellability score
            i = row.Index
            #minimize sellabilty using inverse for minheap
            heap_item = (1/row.Sellability, i, row)
            #add tuple to heap_list
            heap_list.append(heap_item)
        #ADD heap_list to heap dictionary
        heap_dict[category] = heap_list

    #heapify each heap list
    def heapify_dict(dict_of_lists):
        for category in dict_of_lists:
            heapq.heapify(dict_of_lists[category])
        return
    
    heapify_dict(heap_dict)

    #sort each heap
    def sort_heaps(data):
        #create list to store heap 
        sorted_heap = []
        while data:
            sorted_heap.append(heapq.heappop(data))
        return sorted_heap

    #use above function to sort each heap into heap_dict
    for category in heap_dict:
        # copies theheap_dict [category] so as not to sort in place
        sorted_heap = sort_heaps(heap_dict[category].copy()) 
        heap_dict2[category] = sorted_heap

    return heap_dict, heap_dict2

# O(n^2) Algorithm

In [ ]:
# O(N^2 duplicate search algorithm)
# The SKU_to_number algorithm comes from ChatGPT, to optimize the SKU data into unique numbers
# For O(n^2) processing, 2 million rows were not feasible on my computer without optimization
# The way it works is by creating a base 36 numbering system
# 0-9 and A-Z map to a number 0-35. With each additional character, just as in base 10,
# I multiply existing number by base, and add the value of the new character.
# with each SKU as a unique number, I can use the JIT Numba module which speeds up the nested loop considerably
# (this operation was previously impossible on a 2 million large data set on my computer prior.)

def sku_to_number(sku):
    charset = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base = len(charset)
    char_to_val = {c: i for i, c in enumerate(charset)}
    
    sku = sku.upper()
    num = 0
    for c in sku:
        num = num * base + char_to_val[c]
    return num

sku_numeric = np.array([sku_to_number(i)for i in df_numpy])

@njit
def duplicate_search(data):
    # create an empty list to store duplicates
    duplicate_list = []
    # create a counter of duplicates
    duplicates = 0
    length = len(data)
    # nested loop, search for duplicates
    for i in range(length):
        for j in range(i+1,length):
            if data[i] == data[j]:
                #if you find one, add to counter, append to list
                duplicates += 1
                duplicate_list.append(data[i])
    return duplicates, duplicate_list

# Algorithms iterated and timed
- ### for: n = 100_000 -> 2_000_000

In [ ]:
# the data size (to be plotted on x-axis)
plotter_x = []
# the time the n algorithm takes (to be plotted on y-axis)
plotter_y_1 = []
plotter_y_n = []
plotter_y_n2 = []
plotter_y_nlogn = []

for size in range(100_000, 2_000_001, 100_000):
    # create list of data sizes for x-axis:
    plotter_x.append(size)

    # O(1):
    # hash lookup
    # generate a random SKU
    random_SKU = np.random.choice(df_numpy[:size])
    print(f"O(1) complexity, n = {size}:")
    start = time.perf_counter()
    # print the result and the time, then append to time list for O(1)
    print(f"Dictionary lookup of {random_SKU}: {dict_lookup(random_SKU)}")
    print(f"time elapsed: {(total_time := (time.perf_counter() - start))}")
    plotter_y_1.append(total_time)

    # O(n):
    # search for duplicates using a set:
    print(f"O(n) complexity, n = {size}:")
    start = time.perf_counter()
    # print number of duplicates found, then append times to list for O(n)
    print(f"{duplicate_search_set(df_numpy[:size])[0]} duplicates found")
    print(f"time elapsed: { ( total_time := (time.perf_counter() - start) ) }")
    plotter_y_n.append(total_time)

    print(f"O(nlogn) complexity, n = {size}")
    start = time.perf_counter()
    # generate a sorted minheap list for each category
    dict, dict_1 = heapify_categories(df[:size], column = "Category")
    print(f"time elapsed: {(total_time := (time.perf_counter() - start))}")
    plotter_y_nlogn.append(total_time)

    # O(n**2):
    # duplicate search using naive nested for loop
    print(f"O(n^2) complexity, n = {size}:")
    start = time.perf_counter()
    print(f"{duplicate_search(sku_numeric[:size])[0]} duplicates found")
    print(f"time elapsed: {(total_time := (time.perf_counter() - start))}")
    plotter_y_n2.append(round(total_time, 6))

# Plot code

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.plot(np.array(plotter_x), (plotter_y_n2), label = "O(n^2)")
plt.plot(np.array(plotter_x), (plotter_y_n), label = "O(n)")
plt.plot(np.array(plotter_x), (plotter_y_1), label= "O(1)")
plt.plot(np.array(plotter_x), (plotter_y_nlogn), label = "O(nlogn)")
plt.title("Time Complexity")
plt.xlabel("Data Size")
plt.ylabel("Time in seconds")
plt.legend()
plt.show()


# Discussion:

The algorithms produced results as expected with a few caveats:
- O(1), O(n), O(nlogn) took increasingly more time, however they didn't differ a whole lot at small data sizes. At 100,000 data entries, each completed their tasks in under 1 second. However, at 2 million entries, O(nlogn) took around 5 seconds, while O(n) took a third of a second, and O(1) clocked in at a thousandth of a second.
- To gather the times, time.perf_counter was used at the start and finish for each algorithm.
- O(n^2) took longer than the others by a wide margin. Even at 100K entries, it took over 3 seconds. As data grew, n^2 revealed a severe handicap. It was so slow that it became untenable, even for a dataset of only 2 Million. Two optimizations suggested by AI ended up making it feasible (but still slow): the Numba module which "just in time" compiles into machine code, and a string to number function which enabled Numba to work with data in the form of numbers rather than strings.
- For big data sets where waiting days is not possible, choice of algorithm has an important impact. For 2 million data entries, and even with optimizations, the O(n^2) algorithm took north of 20 minutes while O(nlogn) took only 5 seconds. 
- It was suprising to see how useful and quick it is to hash O(1). With one pass across data, hashing provides a map to use in perpetuity for instant access. There was a hashing element involved in every data structure used for each of the algorithms above.
- Mapping from category to sorted min heap took the most effort. The dataframe was separated into a dictionary of smaller dataframes, one per category. Using the smaller dataframes, a dictionary was constructed to key the category to a list of rows, and then from category to min-heap and sorted min-heap of rows. The purpose of the minheap for an e-commerce business, was that if interested in selling a particular product, it would be wise to consider similar but more sellable products in the same category space before making investments (which could result in more money without needing to pivot).
